In [1]:
#wheat leaf
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load and preprocess the dataset
dataset = keras.preprocessing.image_dataset_from_directory(
    'd:\Rice',
    labels='inferred',
    label_mode='binary',
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=(64, 64),
    batch_size=32
)

# Configure the dataset for performance
dataset = dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Define the CNN model
model = keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(dataset, epochs=10)

# Save the model for future use
model.save('Rice_Leaf_model')

Found 802 files belonging to 2 classes.
Using 642 files for training.
Epoch 1/10
21/21 [==============================] - 29s 99ms/step - loss: 0.7794 - accuracy: 0.5093
Epoch 2/10
21/21 [==============================] - 2s 100ms/step - loss: 0.6933 - accuracy: 0.5093
Epoch 3/10
21/21 [==============================] - 2s 99ms/step - loss: 0.6906 - accuracy: 0.5312
Epoch 4/10
21/21 [==============================] - 2s 101ms/step - loss: 0.6865 - accuracy: 0.5514
Epoch 5/10
21/21 [==============================] - 2s 97ms/step - loss: 0.6833 - accuracy: 0.5358
Epoch 6/10
21/21 [==============================] - 2s 95ms/step - loss: 0.6797 - accuracy: 0.5561
Epoch 7/10
21/21 [==============================] - 2s 93ms/step - loss: 0.6825 - accuracy: 0.5592
Epoch 8/10
21/21 [==============================] - 2s 94ms/step - loss: 0.6819 - accuracy: 0.5421
Epoch 9/10
21/21 [==============================] - 2s 95ms/step - loss: 0.6727 - accuracy: 0.5717
Epoch 10/10
21/21 [=================

INFO:tensorflow:Assets written to: Rice_Leaf_model\assets


INFO:tensorflow:Assets written to: Rice_Leaf_model\assets


In [12]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load the saved model
model = keras.models.load_model("C:/Users/sharm/OneDrive/Desktop/Website/Rice_Leaf_model")

# Make predictions on all images in a directory
def predict_leaf_for_directory(directory_path):
    rot=1
    healthy=1
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpg.jpeg") or filename.endswith(".jpg") or filename.endswith(".JPG"):
            image_path = os.path.join(directory_path, filename)
            print(image_path)
            image = keras.preprocessing.image.load_img(image_path, target_size=(64, 64))
            input_arr = keras.preprocessing.image.img_to_array(image)
            input_arr = np.array([input_arr])  # Convert single image to a batch
            input_arr = input_arr / 255.0  # Normalize pixel values
            predictions = model.predict(input_arr)

            if predictions[0] > 0.005:
                label = "Healthy Leaf"
                print(label , predictions)
                healthy = healthy + 1
            else:
                label = "Rot Leaf"
                print(label , predictions)
                rot = rot + 1
    print("rot count " , rot)
    print("Healthy count",healthy)

# Example usage:
directory_path = "D:/Rice/Unhealthy"
predict_leaf_for_directory(directory_path)


D:/Rice/Unhealthy\IMG_20190419_094245.jpg
1/1 [==============================] - 0s 88ms/step
Healthy Leaf [[0.3986026]]
D:/Rice/Unhealthy\IMG_20190419_094254.jpg
1/1 [==============================] - 0s 40ms/step
Healthy Leaf [[0.3970029]]
D:/Rice/Unhealthy\IMG_20190419_094313.jpg
1/1 [==============================] - 0s 24ms/step
Healthy Leaf [[0.39838293]]
D:/Rice/Unhealthy\IMG_20190419_094330.jpg
1/1 [==============================] - 0s 24ms/step
Healthy Leaf [[0.39816922]]
D:/Rice/Unhealthy\IMG_20190419_094850.jpg
1/1 [==============================] - 0s 24ms/step
Healthy Leaf [[0.39684698]]
D:/Rice/Unhealthy\IMG_20190419_095001.jpg
1/1 [==============================] - 0s 24ms/step
Healthy Leaf [[0.39766753]]
D:/Rice/Unhealthy\IMG_20190419_095008.jpg
1/1 [==============================] - 0s 24ms/step
Healthy Leaf [[0.39779723]]
D:/Rice/Unhealthy\IMG_20190419_095012.jpg
1/1 [==============================] - 0s 24ms/step
Healthy Leaf [[0.397682]]
D:/Rice/Unhealthy\IMG_201904

KeyboardInterrupt: 

In [15]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Path to the directory containing rice leaf images
data_dir = 'd:\Rice'

# Set the target size for resizing
target_size = (64, 64)

# Get the list of image file paths
image_paths = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
            image_paths.append(os.path.join(root, file))

# Load and preprocess the images
images = []
labels = []
for image_path in image_paths:
    image = keras.preprocessing.image.load_img(image_path, target_size=target_size)
    image = keras.preprocessing.image.img_to_array(image)
    image = image / 255.0  # Rescale pixel values to [0, 1]
    images.append(image)
    
    # Extract label from the directory name
    label = os.path.basename(os.path.dirname(image_path))
    labels.append(label)

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Create a binary label array (0 for healthy, 1 for diseased)
binary_labels = np.where(labels == 'diseased', 1, 0)

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((images, binary_labels))
dataset = dataset.shuffle(1000).batch(32)

# Define the CNN model
model = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(target_size[0], target_size[1], 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(dataset, epochs=10)

# Save the model for future use
model.save('Rice_Leaf_model')


Epoch 1/10
26/26 [==============================] - 4s 104ms/step - loss: 0.0443 - accuracy: 0.9601
Epoch 2/10
26/26 [==============================] - 3s 101ms/step - loss: 3.1596e-19 - accuracy: 1.0000
Epoch 3/10
26/26 [==============================] - 3s 102ms/step - loss: 1.9939e-20 - accuracy: 1.0000
Epoch 4/10
26/26 [==============================] - 3s 106ms/step - loss: 1.6908e-20 - accuracy: 1.0000
Epoch 5/10
26/26 [==============================] - 3s 100ms/step - loss: 1.6769e-20 - accuracy: 1.0000
Epoch 6/10
26/26 [==============================] - 3s 102ms/step - loss: 1.6745e-20 - accuracy: 1.0000
Epoch 7/10
26/26 [==============================] - 3s 101ms/step - loss: 1.6744e-20 - accuracy: 1.0000
Epoch 8/10
26/26 [==============================] - 3s 99ms/step - loss: 1.6744e-20 - accuracy: 1.0000
Epoch 9/10
26/26 [==============================] - 3s 101ms/step - loss: 1.6744e-20 - accuracy: 1.0000
Epoch 10/10
26/26 [==============================] - 3s 102ms/step - 

INFO:tensorflow:Assets written to: Rice_Leaf_model\assets


INFO:tensorflow:Assets written to: Rice_Leaf_model\assets


In [25]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load the saved model
model = keras.models.load_model("C:/Users/sharm/OneDrive/Desktop/Website/Rice_Leaf_model")
# Make predictions on all images in a directory
def predict_leaf_for_directory(directory_path):
    rot=1
    healthy=1
    pred=[]
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpg.jpeg") or filename.endswith(".jpg") or filename.endswith(".JPG"):
            image_path = os.path.join(directory_path, filename)
            print(image_path)
            image = keras.preprocessing.image.load_img(image_path, target_size=(64, 64))
            input_arr = keras.preprocessing.image.img_to_array(image)
            input_arr = np.array([input_arr])  # Convert single image to a batch
            input_arr = input_arr / 255.0  # Normalize pixel values
            predictions = model.predict(input_arr)
            pred.append(predictions[0][0])
            prednew=float(predictions[0][0])
            print(prednew)
            if prednew < 3.0 and prednew > 7.0 :
                label = "Healthy Leaf"
                print(label , predictions)
                healthy = healthy + 1
            else:
                label = "Rot Leaf"
                print(label , predictions)
                rot = rot + 1
    print("rot count " , rot)
    print("Healthy count",healthy)
    print(pred)
# Example usage:
directory_path = "D:/Rice/Healthy"
predict_leaf_for_directory(directory_path)

D:/Rice/Healthy\IMG_20190419_094746.jpg
1/1 [==============================] - 0s 82ms/step
1.9977015745064656e-23
Rot Leaf [[1.9977016e-23]]
D:/Rice/Healthy\IMG_20190419_094756.jpg
1/1 [==============================] - 0s 26ms/step
6.921728115318415e-23
Rot Leaf [[6.921728e-23]]
D:/Rice/Healthy\IMG_20190419_094925.jpg
1/1 [==============================] - 0s 46ms/step
5.358931300767683e-22
Rot Leaf [[5.3589313e-22]]
D:/Rice/Healthy\IMG_20190419_094932.jpg
1/1 [==============================] - 0s 32ms/step
3.189747828993262e-24
Rot Leaf [[3.1897478e-24]]
D:/Rice/Healthy\IMG_20190419_094943.jpg
1/1 [==============================] - 0s 39ms/step
2.92285132774987e-22
Rot Leaf [[2.9228513e-22]]
D:/Rice/Healthy\IMG_20190419_094948.jpg
1/1 [==============================] - 0s 27ms/step
3.2562975154642905e-24
Rot Leaf [[3.2562975e-24]]
D:/Rice/Healthy\IMG_20190419_095003.jpg
1/1 [==============================] - 0s 41ms/step
5.327356291066435e-23
Rot Leaf [[5.327356e-23]]
D:/Rice/Health

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load the saved model
model = keras.models.load_model("C:/Users/sharm/OneDrive/Desktop/BDA_Project/Rice_Leaf_model")
# Make predictions on all images in a directory
def predict_leaf_for_directory(directory_path):
    rot=1
    healthy=1
    pred=[]
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpg.jpeg") or filename.endswith(".jpg") or filename.endswith(".JPG"):
            image_path = os.path.join(directory_path, filename)
            print(image_path)
            image = keras.preprocessing.image.load_img(image_path, target_size=(64, 64))
            input_arr = keras.preprocessing.image.img_to_array(image)
            input_arr = np.array([input_arr])  # Convert single image to a batch
            input_arr = input_arr / 255.0  # Normalize pixel values
            predictions = model.predict(input_arr)
            pred.append(predictions[0][0])
            prednew=float(predictions[0][0])
            print(prednew)
            predictions= 4.929097582e-22
            prediction = prednew
            formatted_prediction = "{:.1e}".format(prediction)
            formatted_prediction = float(formatted_prediction)
            formatted_prediction1 = str(formatted_prediction)  
            numbers_before_e =float(formatted_prediction1.split('e')[0]) 
            print(numbers_before_e)
            if numbers_before_e < 3.0 or numbers_before_e > 6.0 :
                label = "Healthy Leaf"
                print(label , numbers_before_e)
                healthy = healthy + 1
            else:
                label = "Rot Leaf"
                print(label , numbers_before_e)
                rot = rot + 1
    print("rot count " , rot)
    print("Healthy count",healthy)
    print(pred)
# Example usage:
directory_path = "D:/Rice/unhealthy"
predict_leaf_for_directory(directory_path)

D:/Rice/unhealthy\IMG_20190419_094245.jpg
1/1 [==============================] - 0s 80ms/step
0.5068225860595703
0.51
Healthy Leaf 0.51
D:/Rice/unhealthy\IMG_20190419_094254.jpg
1/1 [==============================] - 0s 34ms/step
0.5071690678596497
0.51
Healthy Leaf 0.51
D:/Rice/unhealthy\IMG_20190419_094313.jpg
1/1 [==============================] - 0s 32ms/step
0.5068634748458862
0.51
Healthy Leaf 0.51
D:/Rice/unhealthy\IMG_20190419_094330.jpg
1/1 [==============================] - 0s 31ms/step
0.506712019443512
0.51
Healthy Leaf 0.51
D:/Rice/unhealthy\IMG_20190419_094850.jpg
1/1 [==============================] - 0s 31ms/step
0.5061678886413574
0.51
Healthy Leaf 0.51
D:/Rice/unhealthy\IMG_20190419_095001.jpg
1/1 [==============================] - 0s 28ms/step
0.5065625309944153
0.51
Healthy Leaf 0.51
D:/Rice/unhealthy\IMG_20190419_095008.jpg
1/1 [==============================] - 0s 25ms/step
0.5059984922409058
0.51
Healthy Leaf 0.51
D:/Rice/unhealthy\IMG_20190419_095012.jpg
1/1 [==

In [31]:
# Assuming your prediction is stored as pred_output
prediction = 4.9033393e-22

formatted_prediction = "{:.1e}".format(prediction)
formatted_prediction = float(formatted_prediction)

print(formatted_prediction)



4.9e-22


In [74]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load the saved model
model = keras.models.load_model("C:/Users/sharm/OneDrive/Desktop/Website/Rice_Leaf_model")

# Function to predict the label of a single leaf image
def predict_leaf(image_path):
    image = keras.preprocessing.image.load_img(image_path, target_size=(64, 64))
    input_arr = keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch
    input_arr = input_arr / 255.0  # Normalize pixel values
    predictions = model.predict(input_arr)
    prediction = predictions[0][0]
    prednew=float(predictions[0][0])
    print(prednew)
    prediction = prednew
    formatted_prediction = "{:.1e}".format(prediction)
    formatted_prediction = float(formatted_prediction)
    formatted_prediction1 = str(formatted_prediction)  
    numbers_before_e =float(formatted_prediction1.split('e')[0]) 
    print(numbers_before_e)
    # Convert prediction to label
    if numbers_before_e < 3.0 or numbers_before_e >6.6 :
        label = "healthy Leaf"
    else:
        label = "unHealthy Leaf"
    
    return label, prediction

# Example usage:
image_path = "D:/Rice/Unhealthy/IMG_20190419_113353.jpg"
label, prediction = predict_leaf(image_path)
print("Label:", label)
print("Prediction:", prediction)


1/1 [==============================] - 0s 67ms/step
6.453753738235965e-23
6.5
Label: unHealthy Leaf
Prediction: 6.453753738235965e-23
